In [46]:
import os
import numpy as np
import pandas as pd
import math
import subprocess
from sklearn.cluster import DBSCAN
import sys
from scipy import stats

def pdb_xyz(pdbline, column):
    if pdbline[0:6]==column:
        x = float('{:.3f}'.format(float(pdbline[31:38])))
        y = float('{:.3f}'.format(float(pdbline[39:46])))
        z = float('{:.3f}'.format(float(pdbline[47:54])))
        xyz = [x, y, z]
        vec_xyz = np.array(xyz)
        return vec_xyz
    else:
        return 'None'
    
def dist_cf(vec_a, vec_b, d_min):
    distance = float(np.linalg.norm(vec_a - vec_b))
    if distance <= d_min:
        return distance
    else:
        return d_min
    
def t_file(filename):
    with open(filename,'w')as file:
        file.truncate(0)

def appr(xxx):
    if xxx < 0:
        return '{:7.03f}'.format(xxx - 0.5)
    else:
        return '{:7.03f}'.format(xxx + 0.5)
        
def lat_gen(inputname, column, outputname):
    t_file('lat.pdb')
    p_num = 0
    with open(inputname,'r')as poc:
        for line in poc:
            if line[0:6] == column:
                p_num += 1
                num_pdb = '{:5}'.format(p_num)
                lxi = math.modf(float(line[31:38]))[1]
                lyi = math.modf(float(line[39:46]))[1]
                lzi = math.modf(float(line[47:54]))[1]
                llx = appr(lxi)
                lly = appr(lyi)
                llz = appr(lzi)
                with open('lat.pdb','a')as lat:
                    print('HETATM'+str(num_pdb)+'      PLA A   1      '+str(llx)+' '+str(lly)+' '+str(llz)+'  1.00 10.00           H', file=lat)
    t_file('lat_ex.txt')
    with open('lat.pdb','r')as poc:
        for line in poc:
            lat_x = float(line[31:38])
            lat_y = float(line[39:46])
            lat_z = float(line[47:54])
            for j in range(3):
                x = round(float(j-1.0),1)
                for k in range(3):
                    y = round(float(k-1.0),1)
                    for l in range(3):
                        z = round(float(l-1.0),1)
                        with open('lat_ex.txt','a')as exp:
                            print(lat_x+x, lat_y+y, lat_z+z, file=exp)
    df = pd.read_csv('lat_ex.txt', sep=" ",header=None)
    dup = df.drop_duplicates()
    t_file(outputname)
    p_num = 0
    for item in zip(dup[0],dup[1],dup[2]):
        p_num += 1
        num_pdb = '{:5}'.format(p_num)
        one_x = '{:7.03f}'.format(float(item[0]))
        one_y = '{:7.03f}'.format(float(item[1]))
        one_z = '{:7.03f}'.format(float(item[2]))
        with open (outputname,'a')as poc:
            print('HETATM'+num_pdb+'      PLA A   1     '+one_x+' '+one_y+' '+one_z+'  1.00 10.00           H', file=poc)
    subprocess.call(['rm', 'lat.pdb'])
    subprocess.call(['rm', 'lat_ex.txt'])
            
workdir = os.getcwd()
os.chdir(workdir)


        
#インプットの設定
print('Enter common name of complex.')
print('ex) If you have complex files named "prot_1.pdb ~ prot_X.pdb", please type "prot_"')
print('(Note that only pdb-format can be specified for complex file)')
common_pro = input('>>')
print('Enter common name of ligand.')
print('ex) If you have ligand files named "lig_1.pdb ~ lig_X.pdb", please type "lig_"')
print('(Note that only pdb-format can be specified for ligand file)')
common_lig = input('>>')
print('Enter the number of trajectories.')
print('ex) If you have 10 trajectories, please type "10" in Arabic numerals')
loop_no = input('>>')
try:
    int(loop_no)
    l_int = True
except ValueError:
    l_int = False
    print('Input Error: Type the number of trajectories in Arabic numerals.')
    sys.exit()
for i in range(int(loop_no)):
    lno=str(i+1)
    pro_exist = os.path.isfile('./'+common_pro+lno+'.pdb')
    if not pro_exist:
        print('Input Error: "./'+common_pro+lno+'.pdb" does not exist in the directory.')
        sys.exit()
    lig_exist = os.path.isfile('./'+common_lig+lno+'.pdb')
    if not lig_exist:
        print('Input Error: "./'+common_lig+lno+'.pdb" does not exist in the directory.')
        sys.exit()
print('All input files were checked.')

t_file('detected_point.pdb')
t_file('next_poc.pdb')


for i in range(int(loop_no)):
    lno = str(i+1)
    subprocess.call(['mkdir', './tra_'+lno])
    subprocess.call(['cp', common_pro+lno+'.pdb', './tra_'+lno])
    subprocess.call(['cp', common_lig+lno+'.pdb', './tra_'+lno])

for loopno in range(int(loop_no)):
    lno = str(loopno+1)
    os.chdir(workdir)
    os.chdir('./tra_'+lno+'/')
    protein = common_pro+lno+'.pdb'
    name = common_pro+lno
    ligand_ori = common_lig+lno+'.pdb'
    with open(ligand_ori)as lori:
        content = lori.read()
    content = content.replace('ATOM  ', 'HETATM')
    with open(common_lig+lno+'_fixed.pdb','w')as wri:
        wri.write(content)
    ligand_ori = common_lig+lno+'_fixed.pdb'
    ligname = common_lig+lno+'_fixed'
    t_file(ligname+'_noh.pdb')
    with open(ligand_ori,'r')as lig:
        for line in lig:
            if line[13:14]!='H':
                with open(ligname+'_noh.pdb','a')as ligh:
                    print(line, end='', file=ligh)
    ligand = ligname+'_noh.pdb'
    defpoc = name+'_pockets.pqr'
    trimpoc = 'trim_pocket.pqr'
    
    #トリミング＋クラスタリングの設定値
    #リガンド結合ポケットの選択範囲(リガンド原子からの距離)
    lig_bind_poc = float(10)
    #トリミング＋クラスタリングのしきい値
    d = 1.53
    num = str(4)


    #fpocketの操作
    subprocess.call(['rm', '-r', name+'_out'])
    subprocess.call(['rm', name+'_out.pdb'])
    subprocess.call(['rm', name+'_pockets.pqr'])
    subprocess.call(['fpocket', '-f', protein, '-i', '0'])
    subprocess.call(['cp', name+'_out/'+name+'_out.pdb', './'])
    subprocess.call(['cp', name+'_out/'+name+'_pockets.pqr', './'])
    print('【tra_'+lno+' part1】fpocket-done')

    #fpocketのオリジナルポケット数(※≠アルファ球数)を取得
    with open(name+'_pockets.pqr','r')as poc:
        ori_poc_num = int(str(poc.readlines()[-3:-2])[22:28])

    #リガンド周辺のアルファ球を1つのligand_binding_pocket.pqrへ
    t_file('ligand_binding_pocket.pqr')
    for j in range(ori_poc_num):
        distance_min = float(100000)
        t_file('poc_tmp.pqr')
        with open(defpoc,'r')as poc:
            for pline in poc:
                pnum = str(j+1)
                if pline.split()[0] == 'ATOM' and int(pline[22:28])==int(pnum):
                    with open('poc_tmp.pqr','a')as tmp:
                        print(pline, end='',file=tmp)
                elif pline[22:28]==int(pnum)+1:
                    break
        with open('poc_tmp.pqr','r')as tmp:
            for pline in tmp:
                vec_1 = pdb_xyz(pline,'ATOM  ')
                if str(vec_1) != 'None':
                    with open(ligand,'r')as lig:
                        for lline in lig:
                            vec_2 = pdb_xyz(lline,'HETATM')
                            if str(vec_2) != 'None':
                                dis_min = dist_cf(vec_1, vec_2, distance_min)
                                distance_min = dis_min
        if distance_min <= lig_bind_poc:
            with open('poc_tmp.pqr','r')as tmp:
                for line in tmp:
                    with open('ligand_binding_pocket.pqr','a')as pocket:
                        print(line, end='', file=pocket)
    subprocess.call(['rm', 'poc_tmp.pqr'])

    #トリミング処理
    loop = 0
    for j in range(1000):
        if j == 0:
            before = 'ligand_binding_pocket.pqr'
        else:
            before = str(loop)+'.pqr'
        loop += 1
        after = str(loop)+'.pqr'
        t_file(after)
        with open(before,'r')as bef1:
            for line1 in bef1:
                vec_1 = pdb_xyz(line1,'ATOM  ')
                if str(vec_1) != 'None':
                    count_in = 0
                    with open(before,'r')as bef2:
                        for line2 in bef2:
                            vec_2 = pdb_xyz(line2,'ATOM  ')
                            if str(vec_2) != 'None':
                                dist = float(np.linalg.norm(vec_1-vec_2))
                                if dist != 0 and dist<=float(d):
                                    count_in += 1
                    if count_in >= int(num):
                        with open(after,'a')as aft:
                            print(line1, end='', file=aft)
        count_b = sum([1 for _ in open(before)])
        count_a = sum([1 for _ in open(after)])
        if j != 0:
            subprocess.call(['rm', before])
        if count_b == count_a:
            subprocess.call(['mv', after, 'trim_pocket.pqr'])
            break
    print('【tra_'+lno+' part2】trimming-done')

    #DVO計算用、格子生成
    lat_gen(trimpoc, 'ATOM  ', 'poc_lat.pdb')
    lat_gen(ligand, 'HETATM', 'lig_lat.pdb')
    print('【tra_'+lno+' part3】lattice generation-done')

    #アルファ球格子ーリガンド格子
    t_file('poc_surp.pdb')
    t_file('poc_next.pdb')
    with open('poc_lat.pdb','r')as pla:
        for pline in pla:
            px = float(pline[31:38])
            py = float(pline[39:46])
            pz = float(pline[47:54])
            dupl = 0
            with open('lig_lat.pdb','r')as lla:
                for lline in lla:
                    lx = float(lline[31:38])
                    ly = float(lline[39:46])
                    lz = float(lline[47:54])
                    if px==lx and py==ly and pz==lz:
                        dupl += 1
                        break
                if dupl==0:
                    with open('poc_surp.pdb','a')as sur:
                        print(pline, end='', file=sur)

    #残りアルファ球の作成
    num = 0
    with open(trimpoc,'r')as poc:
        for l in poc:
            lxi = math.modf(float(l[31:38]))[1]
            llx = appr(lxi)
            lyi = math.modf(float(l[39:46]))[1]
            lly = appr(lyi)
            lzi = math.modf(float(l[47:54]))[1]
            llz = appr(lzi)
            check = 0
            with open('lig_lat.pdb','r')as lla:
                for lline in lla:
                    llax = float(lline[31:38])
                    llay = float(lline[39:46])
                    llaz = float(lline[47:54])
                    #print(llax,llay,llaz)
                    if float(llx) == float(llax) and float(lly) == float(llay) and float(llz) == float(llaz):
                        check += 1
                        break
            if check == 0:
                num += 1
                num_pdb = '{:5}'.format(num)
                with open('poc_next.pdb','a')as pro:
                    print(l, end='', file=pro)

    #クラスタリングDBSCAN
    t_file('cluster.pdb')
    data = []
    with open('poc_next.pdb','r')as pro:
        for line in pro:
            lst = [float(line[31:38]), float(line[39:46]), float(line[47:54])]
            data.append(lst)
    db = DBSCAN(eps=1.53, min_samples=5).fit(data)
    labels = db.labels_
    for i in range(len(data)):
        with open('poc_next.pdb','r')as pro:
            p = pro.readlines()[i]
            if str(labels[i]) != str(-1):
                with open('cluster.pdb','a')as cls:
                    print(p[0:20], str('{:5}'.format(labels[i])), p[27:], end='', file=cls)
    print('【tra_'+lno+' part4】clustering-done')

    #リガンドの伸長点限定
    t_file('R-candidate.pdb')
    check = 0
    with open(ligand_ori,'r')as lig1:
        for line1 in lig1:
            if line1[13:14]=='H':
                check += 1
                break
        if check == 0:
            with open(ligand_ori,'r')as lig2:
                for line2 in lig2:
                    with open('R-candidate.pdb','a')as cand:
                        print(line2, end='', file=cand)
            print('Warning! "'+common_lig+lno+'.pdb" have any hydrogen bond!')
    if check != 0:
        with open(ligand_ori,'r')as lig1:
            for line1 in lig1:
                if line1[0:6]=='HETATM' and line1[13:14]!='H':
                    with open(ligand_ori,'r')as lig2:
                        for line2 in lig2:
                            if line2[0:6]=='CONECT' and int(line1[6:11])==int(line2[6:11]):
                                lst=line2.split()[2:]
                                for con in lst:
                                    with open(ligand_ori,'r')as lig3:
                                        for line3 in lig3:
                                            if line3[0:6]=='HETATM' and int(line3[6:11])==int(con):
                                                if line3[13:14]=='H':
                                                    with open('R-candidate.pdb','a')as can:
                                                        print(line1, end='', file=can)
    df = pd.read_csv('R-candidate.pdb',header=None)
    dup = df.drop_duplicates()
    dup.to_csv('R-candidate.pdb',sep=",", header=False, index=False)

    #始点Rの指定
    dist = float(5000)
    with open('R-candidate.pdb','r')as lig:
        for line in lig:
            vec_lig = np.array([float(line[31:38]), float(line[39:46]), float(line[47:54])])
            distance_min = float(5000)
            with open('cluster.pdb','r')as cls:
                for aline in cls:
                    vec_alp = np.array([float(aline[31:38]), float(aline[39:46]), float(aline[47:54])])
                    distance = float(np.linalg.norm(vec_lig - vec_alp))
                    if distance <= distance_min:
                        distance_min = distance
                        pocket = aline
            if distance_min <= dist:
                d_line = line
                d_pocket = pocket
                dist = distance_min
    t_file('R.pdb')
    with open('R.pdb','a')as rrr:
        print(d_line, end='', file=rrr)
    t_file('next_pocket.pdb')
    with open('cluster.pdb','r')as cls:
        for line in cls:
            if int(line[22:26])==int(d_pocket[22:26]):
                with open('next_pocket.pdb','a')as nxt:
                    print(line, end='', file=nxt)
                with open('../next_poc.pdb','a')as nxt:
                    print(line, end='', file=nxt)
    with open('../detected_point.pdb','a')as dtp:
        print(d_line, end='', file=dtp)
    print('【tra_'+lno+' part5】R detection-done')

#Rの最頻値
os.chdir(workdir)
r = []
with open('detected_point.pdb','r')as dtp:
    for line in dtp:
        r.append(int(line[6:11]))

uniqs, counts = np.unique(r, return_counts=True)
mode = uniqs[counts == np.amax(counts)]
if len(mode)==1:
    det_R = mode[np.argmax(comp)]
    print('detected-R is "'+str(mode[0])+'"')
else:
    comp = []
    for i in mode:
        alp_num = 0
        number = str(i)
        for j in range(len(r)):
            if str(r[j])==number:
                a_num = 0
                with open(workdir+'/tra_'+str(j+1)+'/next_pocket.pdb','r')as nxt:
                    for line in nxt:
                        a_num += 1
                if alp_num <= a_num:
                    alp_num = a_num
        comp.append(alp_num)
    det_R = mode[np.argmax(comp)]
    print('detected-R is "'+str(mode[np.argmax(comp)])+'"')


Enter common name of complex.
ex) If you have complex files named "prot_1.pdb ~ prot_X.pdb", please type "prot_"
(Note that only pdb-format can be specified for complex file)
>>prot_
Enter common name of ligand.
ex) If you have ligand files named "lig_1.pdb ~ lig_X.pdb", please type "lig_"
(Note that only pdb-format can be specified for ligand file)
>>lig_
Enter the number of trajectories.
ex) If you have 10 trajectories, please type "10" in Arabic numerals
>>10
All input files were checked.
【tra_1 part1】fpocket-done
【tra_1 part2】trimming-done
【tra_1 part3】lattice generation-done
【tra_1 part4】clustering-done
【tra_1 part5】R detection-done
【tra_2 part1】fpocket-done
【tra_2 part2】trimming-done
【tra_2 part3】lattice generation-done
【tra_2 part4】clustering-done
【tra_2 part5】R detection-done
【tra_3 part1】fpocket-done
【tra_3 part2】trimming-done
【tra_3 part3】lattice generation-done
【tra_3 part4】clustering-done
【tra_3 part5】R detection-done
【tra_4 part1】fpocket-done
【tra_4 part2】trimming-done
【tr